In [1]:
import sympy as sym
from sympy.printing import ccode

In [2]:
singvar_functs  = []
singvar_names   = []
multivar_functs = []

# Single variable functions:

# Inverse trigonometric.
singvar_functs.append( sym.asin  ), singvar_names.append("asin"     )
singvar_functs.append( sym.acos  ), singvar_names.append("acos"     )
singvar_functs.append( sym.atan  ), singvar_names.append("atan"     )

# Inverse Hyperbolic
singvar_functs.append( sym.asinh ), singvar_names.append("asinh"    )
singvar_functs.append( sym.acosh ), singvar_names.append("acosh"    )
singvar_functs.append( sym.atanh ), singvar_names.append("atanh"    )

# Error function

singvar_functs.append( sym.erf ), singvar_names.append("erf"    )


# Slower functions.
singvar_functs.append( sym.tan   ), singvar_names.append("tan"      )
singvar_functs.append( sym.tanh  ), singvar_names.append("tanh"     )

# Special functions.
# Depends on multiple inputs.

# singvar_functs.append( sym.log10 ), singvar_names.append("log10"    )
# singvar_functs.append( sym.logb  ), singvar_names.append("logb"     )

# Multivariable functions.
# multivar_functs.append( sym.power ), multivar_names.append("power"    ) # x^y, x variable, y variable.
# multivar_functs.append( sym.atan2 ), multivar_names.append("atan2"    ) # atan2(x,y), x variable, y variable.
# multivar_functs.append(    x/y    ), multivar_names.append("atan2"    ) # atan2(x,y), x variable, y variable.



(None, None)

In [3]:
str_funct_start = "// ****************************************************************************************************\n"
str_funct_end   = "// ----------------------------------------------------------------------------------------------------\n"
tab_sep = " "*4 # Four spaces.
endl="\n"

special_functs = ['tan','tanh']

def write_tan_funct(string_out, fx, x, deriv_order, level):
    string_out += tab_sep*level
    string_out += "ord_t i;"+endl
    string_out += tab_sep*level
    string_out += "coeff_t tx = tan(x0);"+endl

    string_out += tab_sep*level
    string_out += "coeff_t tx2 = pow(tx,2);"+endl

    # Write function evaluation.

    string_out += tab_sep*level
    string_out += "derivs[0]="
    string_out += ccode(fx).replace(" ","").replace("tan(x0)","tx").replace("pow(tx,2)","tx2")

    string_out += ";"+endl

    string_out += tab_sep*level
    string_out += "for(i=1;i<=order;i++){"+endl
    level += 1

    string_out += tab_sep*level
    string_out += "switch(i){"+endl
    level+=1    

    for order in range(1,deriv_order+1):

        print(" -- Writing derivative ",order)
        string_out += tab_sep*level
        string_out+="case "+str(order)+":"+endl
        level+=1
        string_out += tab_sep*level
        string_out += "derivs["+str(order)+"]="
        fx = fx.diff(x)
        
        code = ccode(fx.evalf())
        code = code.replace(" ","")
        code = code.replace("tan(x0)","tx")
        code = code.replace("pow(tx,2)","tx2")
        
        code = code.replace(".0+",".+")
        code = code.replace(".0-",".-")
        code = code.replace(".0*",".*")
        code = code.replace(".0/","./")
        code = code.replace(".0,",".,")
        
        string_out += code
        string_out += ";"+endl
        level-=1

    # end for 


    # ending SWITCH.
    level-=1
    string_out += tab_sep*level
    string_out += "}"+endl


    # ending FOR.
    level-=1
    string_out += tab_sep*level
    string_out += "}"+endl
    
    return string_out
    
    
def write_tanh_funct(string_out, fx, x, deriv_order, level):

    
    string_out += tab_sep*level
    string_out += "ord_t i;"+endl
    string_out += tab_sep*level
    string_out += "coeff_t tx = tanh(x0);"+endl

    string_out += tab_sep*level
    string_out += "coeff_t tx2 = pow(tx,2);"+endl

    # Write function evaluation.

    string_out += tab_sep*level
    string_out += "derivs[0]="
    string_out += ccode(fx).replace(" ","").replace("tanh(x0)","tx").replace("pow(tx,2)","tx2")

    string_out += ";"+endl

    string_out += tab_sep*level
    string_out += "for(i=1;i<=order;i++){"+endl
    level += 1

    string_out += tab_sep*level
    string_out += "switch(i){"+endl
    level+=1    

    for order in range(1,deriv_order+1):

        print(" -- Writing derivative ",order)
        string_out += tab_sep*level
        string_out+="case "+str(order)+":"+endl
        level+=1
        string_out += tab_sep*level
        string_out += "derivs["+str(order)+"]="
        fx = fx.diff(x)
        
        code = ccode(fx.evalf())
        code = code.replace(" ","")
        code = code.replace("tanh(x0)","tx")
        code = code.replace("pow(tx,2)","tx2")
        
        code = code.replace(".0+",".+")
        code = code.replace(".0-",".-")
        code = code.replace(".0*",".*")
        code = code.replace(".0/","./")
        code = code.replace(".0,",".,")
        code = code.replace(".0;",".;")
        code = code.replace(".0)",".)")
        
        string_out += code
        string_out += ";"+endl
        level-=1

    # end for 


    # ending SWITCH.
    level-=1
    string_out += tab_sep*level
    string_out += "}"+endl


    # ending FOR.
    level-=1
    string_out += tab_sep*level
    string_out += "}"+endl

    return string_out
    
    
def write_function(string_out, function, funct_name, deriv_order):
    
    x = sym.symbols('x0')
    fx = function(x)
    
    level = 0
    
    # Write initialization
    string_out += "\n"
    string_out += str_funct_start
    
    string_out += "void der_r_"+funct_name+"(coeff_t x0, ord_t order, coeff_t* derivs){" + endl
    level+=1
    
    # declarations?
    string_out += endl # Space.
    
    if (funct_name in special_functs):
        if funct_name == 'tan':

            string_out = write_tan_funct(string_out, fx, x, deriv_order,level)
            
        elif funct_name == 'tanh':
            
            string_out = write_tanh_funct(string_out, fx, x, deriv_order, level)
            
        # end if 
    else:
        string_out += tab_sep*level
        string_out += "ord_t i;"+endl
        string_out += tab_sep*level
        string_out += "coeff_t x2 = pow(x0,2);"+endl

        # Write function evaluation.

        string_out += tab_sep*level
        string_out += "derivs[0]="
        string_out += ccode(fx).replace(" ","")

        string_out += ";"+endl

        string_out += tab_sep*level
        string_out += "for(i=1;i<=order;i++){"+endl
        level += 1

        string_out += tab_sep*level
        string_out += "switch(i){"+endl
        level+=1    

        for order in range(1,deriv_order+1):

            print(" -- Writing derivative ",order)
            string_out += tab_sep*level
            string_out+="case "+str(order)+":"+endl
            level+=1
            string_out += tab_sep*level
            string_out += "derivs["+str(order)+"]="
            fx = fx.diff(x)
            
            code = ccode(fx.evalf())
            code = code.replace(" ","")
            code = code.replace("pow(x0,2)","x2")

            code = code.replace(".0+",".+")
            code = code.replace(".0-",".-")
            code = code.replace(".0*",".*")
            code = code.replace(".0/","./")
            code = code.replace(".0,",".,")

            string_out += code
            string_out += ";"+endl
            level-=1

        # end for 


        # ending SWITCH.
        level-=1
        string_out += tab_sep*level
        string_out += "}"+endl


        # ending FOR.
        level-=1
        string_out += tab_sep*level
        string_out += "}"+endl
        
    # end if 
    
    string_out += "}"+endl
    string_out += str_funct_end
    
    return string_out
# end

In [4]:
string=""
deriv_order = 10
for i in range(len(singvar_functs)):
    print("Writing the derivatives of function: "+singvar_names[i])
    string = write_function(string, singvar_functs[i], singvar_names[i], deriv_order)
    
    # Write progress.
    filename = "function_derivatives_test3.c"
    f = open(filename,'w')
    f.write(string)
    f.close()
# filename = "function_derivatives_test2.c"
# f = open(filename,'w')
# f.write(string)
# f.close()

Writing the derivatives of function: asin
 -- Writing derivative  1
 -- Writing derivative  2
 -- Writing derivative  3
 -- Writing derivative  4
 -- Writing derivative  5
 -- Writing derivative  6
 -- Writing derivative  7
 -- Writing derivative  8
 -- Writing derivative  9
 -- Writing derivative  10
Writing the derivatives of function: acos
 -- Writing derivative  1
 -- Writing derivative  2
 -- Writing derivative  3
 -- Writing derivative  4
 -- Writing derivative  5
 -- Writing derivative  6
 -- Writing derivative  7
 -- Writing derivative  8
 -- Writing derivative  9
 -- Writing derivative  10
Writing the derivatives of function: atan
 -- Writing derivative  1
 -- Writing derivative  2
 -- Writing derivative  3
 -- Writing derivative  4
 -- Writing derivative  5
 -- Writing derivative  6
 -- Writing derivative  7
 -- Writing derivative  8
 -- Writing derivative  9
 -- Writing derivative  10
Writing the derivatives of function: asinh
 -- Writing derivative  1
 -- Writing derivative

In [ ]:
A = "tan(x0)*pow(tan(x0),2)"
A.replace("tan(x0)","tx")

In [5]:
1exponent = 1.0/2.0

base = "x0"

factor = 1
order = 10
for i in range(order + 1):
    factor = factor * exponent
    exponent = exponent - 1
    
    print("order =    ",i)
    print("factor =   ",factor)
    print("exponent = ",exponent)
    print()


order =     0
factor =    0.5
exponent =  -0.5

order =     1
factor =    -0.25
exponent =  -1.5

order =     2
factor =    0.375
exponent =  -2.5

order =     3
factor =    -0.9375
exponent =  -3.5

order =     4
factor =    3.28125
exponent =  -4.5

order =     5
factor =    -14.765625
exponent =  -5.5

order =     6
factor =    81.2109375
exponent =  -6.5

order =     7
factor =    -527.87109375
exponent =  -7.5

order =     8
factor =    3959.033203125
exponent =  -8.5

order =     9
factor =    -33651.7822265625
exponent =  -9.5

order =     10
factor =    319691.93115234375
exponent =  -10.5



In [9]:
945/64

14.765625

In [3]:
x = sym.symbols("x")
f = 2**x

In [4]:
f.diff(x)

2**x*log(2)

In [5]:
f.diff(x,2)

2**x*log(2)**2

In [11]:
f.diff(x,10).doit().expand().evalf()

0.0256008632895631*2.0**x

In [13]:
help(f.evalf)

Help on method evalf in module sympy.core.evalf:

evalf(n=15, subs=None, maxn=100, chop=False, strict=False, quad=None, verbose=False) method of sympy.core.power.Pow instance
    Evaluate the given formula to an accuracy of n digits.
    Optional keyword arguments:
    
        subs=<dict>
            Substitute numerical values for symbols, e.g.
            subs={x:3, y:1+pi}. The substitutions must be given as a
            dictionary.
    
        maxn=<integer>
            Allow a maximum temporary working precision of maxn digits
            (default=100)
    
        chop=<bool>
            Replace tiny real or imaginary parts in subresults
            by exact zeros (default=False)
    
        strict=<bool>
            Raise PrecisionExhausted if any subresult fails to evaluate
            to full accuracy, given the available maxprec
            (default=False)
    
        quad=<str>
            Choose algorithm for numerical quadrature. By default,
            tanh-sinh quad

In [17]:

f = sym.tan(x)
print(f)

tan(x)


In [15]:
df = f.diff(x)
print(df)

tan(x)**2 + 1


In [19]:
ddf = df.diff(x)
print(ddf)

(2*tan(x)**2 + 2)*tan(x)


In [20]:
val = sym.symbols("df")
print(ddf.subs(df,val))

(2*tan(x)**2 + 2)*tan(x)


In [ ]:
dddf = ddf.diff(x)
print()

In [22]:
help(sym.diff)

Help on function diff in module sympy.core.function:

diff(f, *symbols, **kwargs)
    Differentiate f with respect to symbols.
    
    This is just a wrapper to unify .diff() and the Derivative class; its
    interface is similar to that of integrate().  You can use the same
    shortcuts for multiple variables as with Derivative.  For example,
    diff(f(x), x, x, x) and diff(f(x), x, 3) both return the third derivative
    of f(x).
    
    You can pass evaluate=False to get an unevaluated Derivative class.  Note
    that if there are 0 symbols (such as diff(f(x), x, 0), then the result will
    be the function (the zeroth derivative), even if evaluate=False.
    
    Examples
    
    >>> from sympy import sin, cos, Function, diff
    >>> from sympy.abc import x, y
    >>> f = Function('f')
    
    >>> diff(sin(x), x)
    cos(x)
    >>> diff(f(x), x, x, x)
    Derivative(f(x), (x, 3))
    >>> diff(f(x), x, 3)
    Derivative(f(x), (x, 3))
    >>> diff(sin(x)*cos(y), x, 2, y, 2)
   

In [24]:
sym.geometry.util.idiff(f,x)

TypeError: idiff() missing 1 required positional argument: 'x'